# Model I/O

Different input and output module for the LLM

融合课程还有cookbook等写的一些Toy Example

In [2]:
# set the init envs 
import os, sys
import openai
import langchain
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) 


openai.api_base = os.environ["OPENAI_API_BASE"] # 换成代理，一定要加v1
openai.api_key = os.environ["OPENAI_API_KEY"]

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

llm = OpenAI(
    temperature=0.8,
    model_name="gpt-3.5-turbo",
)
chat = ChatOpenAI(
    temperature=0.0,
    model_name = "gpt-3.5-turbo",
)
def get_completion(prompt, model="gpt-3.5-turbo"):
    
    messages = [{"role": "user", "content": prompt}]
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]

from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

chat = ChatOpenAI(
    model_name = "gpt-3.5-turbo",
    temperature=0.0
    )
print(chat)

llm = OpenAI(
    model_name = "gpt-3.5-turbo",
    temperature = 0.0
)
print(llm)

cache=None verbose=False callbacks=None callback_manager=None tags=None metadata=None client=<class 'openai.api_resources.chat_completion.ChatCompletion'> model_name='gpt-3.5-turbo' temperature=0.0 model_kwargs={} openai_api_key='sk-S0TKP0Sxk8H8VirL7a912cAf06314e46AfB0348eD40183B5' openai_api_base='https://openkey.cloud/v1' openai_organization='' openai_proxy='' request_timeout=None max_retries=6 streaming=False n=1 max_tokens=None tiktoken_model_name=None
OpenAIChat
Params: {'model_name': 'gpt-3.5-turbo', 'temperature': 0.0}


d:\Anaconda3\envs\LLM\lib\site-packages\langchain\llms\openai.py:201: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
d:\Anaconda3\envs\LLM\lib\site-packages\langchain\llms\openai.py:786: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


## Input Modules examples 

### 0. simple input for the model 

consider LLM as a powerful language model.

In [5]:
get_completion("1+1是什么？")

'1+1等于2。'

### 1. the prompt input for the model 



In [8]:
# 美式英语 + 平静、尊敬的语调
style = """American English \
in a calm and respectful tone
"""

customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)


# 普通话 + 平静、尊敬的语调
style = """正式普通话 \
用一个平静、尊敬的语调
"""

# 要求模型根据给出的语调进行转化
prompt = f"""把由三个反引号分隔的文本\
翻译成一种{style}风格。
文本: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```

把由三个反引号分隔的文本翻译成一种正式普通话 用一个平静、尊敬的语调
风格。
文本: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [9]:
response = get_completion(prompt)

### 3. PromptTemplate 


use the python str class as input 

automatically replace variable as input str and feed the template into the llm

In [3]:
from langchain import PromptTemplate
from langchain.prompts import ChatPromptTemplate

In [4]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""


prompt_template = ChatPromptTemplate.from_template(template_string)

print(prompt_template.messages[0].prompt.input_variables)

['style', 'text']


In [5]:
customer_style = """American English \
in a calm and respectful tone
"""

customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [6]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

print(type(customer_messages))
print(type(customer_messages[0]))
print(customer_messages[0])

<class 'list'>
<class 'langchain.schema.messages.HumanMessage'>
content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n" additional_kwargs={} example=False


In [8]:
customer_response = chat(customer_messages)

print(customer_response.content)

I'm really frustrated that my blender lid flew off and made a mess of my kitchen walls with smoothie! And to make things even worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, my friend!


In [10]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""



service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [11]:
service_response = chat(service_messages)
print(service_response.content)

Ahoy there, matey! I regret to inform ye that the warranty be not coverin' the costs o' cleanin' yer galley, as 'tis yer own fault fer misusin' yer blender by forgettin' to secure the lid afore startin' it. Aye, tough luck, me heartie! Fare thee well!


### 4. Explainable language

In [12]:
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    HumanMessage
)
from langchain.chains import LLMChain


# llm = 

human_template="Show me the HEX code of color {color_name}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([human_message_prompt])
chain = LLMChain(llm=llm, prompt=chat_prompt)

chain.run("RED")

'The HEX code for the color red is #FF0000.'

In [13]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Show me the HEX code of color {color_name}")

chain = prompt | llm

chain.invoke({"color_name": "RED"})


'The HEX code for the color red is #FF0000.'

In [15]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

chain = prompt | llm | StrOutputParser()

chain.invoke({"color_name": "RED"})

'The HEX code for the color red is #FF0000.'

In [18]:
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

functions = [
    {
      "name": "save_color_code",
      "description": "Save the HEX code of color and its name",
      "parameters": {
        "type": "object",
        "properties": {
          "hex_code": {
            "type": "string",
            "description": "The HEX code of the color"
          },
          "color": {
            "type": "string",
            "description": "The color name"
          }
        },
        "required": ["hex_code", "color"]
      }
    }
  ]
chain = prompt | chat.bind(function_call = {"name": "save_color_code"}, functions = functions) | JsonOutputFunctionsParser()

chain.invoke({"color_name": "RED"})

{'hex_code': '#FF0000', 'color': 'RED'}

### 4. Input with dataloder
